In [27]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
# import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import csv

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

In [30]:
train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
train_data  = datasets.ImageFolder(train_dir, transform = transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [31]:
# Load pretrained Convnext_tiny
convnext_model = models.convnext_tiny(weights= None).to(device)
convnext_model.classifier[2] = nn.Linear(768,50)
convnext_model.load_state_dict(torch.load('/kaggle/input/cnt-weights-csv/weights15.pth', map_location=torch.device('cpu')))
convnext_model.to(device)
convnext_model.eval()  # Set to evaluation mode

# Load pretrained Resnet
resnet_model = models.resnet18(weights= None).to(device)
resnet_model.fc = nn.Linear(512,50)
resnet_model.load_state_dict(torch.load('/kaggle/input/resnet18-weights-csv/weights20.pth', map_location=torch.device('cpu')))
resnet_model.to(device)
resnet_model.eval()  # Set to evaluation mode


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
class Ensemble(nn.Module):
    def __init__(self, models):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList(models)  # Convert list of models to ModuleList

    def forward(self, x):
        outputs = [model(x) for model in self.models]
        return torch.stack(outputs).mean(0)

# Define the ensemble model
ensemble_model = Ensemble([resnet_model, convnext_model]).to(device)

# Now ensemble_model.parameters() should contain the parameters of the models in the ensemble

# Define loss criterion and optimizer for ensemble model
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)


In [33]:
test_dir = '/kaggle/input/iith-dl-contest-2024/test'
test_data  = datasets.ImageFolder(test_dir, transform = transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [34]:
import numpy as np
import csv

In [35]:
# Reverse the class to index mapping to index to class for prediction interpretation
classes = train_data.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in classes.items()}

In [36]:
# List to store outputs
outputs_list = []

# Process images and gather predictions
for images, _ in tqdm(test_loader):
    images = images.to(device)
    outputs = ensemble_model(images)
    _, predicted = torch.max(outputs, dim=1)
    outputs_list.append(predicted)

# Concatenate all predictions into a single tensor
outputs = torch.cat(outputs_list).cpu().numpy()

100%|██████████| 1199/1199 [03:19<00:00,  6.02it/s]


In [37]:
import numpy as np

# Convert class indices to class names
predicted_classes = np.array([idx_to_class[idx] for idx in outputs], dtype=object)

# Generate image file names
file_names = [f"{i}.JPEG" for i in range(len(predicted_classes))]

file_names = np.sort(file_names)

In [38]:
# Combine file names and predicted classes
table = np.column_stack((file_names, predicted_classes))

# Print table (optional, can be commented out in production)
print(table)

[['0.JPEG' 'n02808440']
 ['1.JPEG' 'n02480495']
 ['10.JPEG' 'n02226429']
 ...
 ['9997.JPEG' 'n02480495']
 ['9998.JPEG' 'n02395406']
 ['9999.JPEG' 'n01774750']]


In [39]:
# Write results to CSV file
with open('submissionI 111.csv', 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(['ID', 'Category'])
    wr.writerows(table)